In [0]:
from pyspark.sql.functions import count,month, year,round
from pyspark.sql.functions import col,concat,lit,lag
from pyspark.sql.window import Window
from pyspark.sql.functions import sum,first,date_format, to_date,coalesce

In [0]:
configs = {"fs.azure.account.auth.type": "OAuth",
"fs.azure.account.oauth.provider.type": "org.apache.hadoop.fs.azurebfs.oauth2.ClientCredsTokenProvider",
"fs.azure.account.oauth2.client.id": "",
"fs.azure.account.oauth2.client.secret": '',
"fs.azure.account.oauth2.client.endpoint": "https://login.microsoftonline.com/tenant_id/oauth2/token"}


dbutils.fs.mount(
source = "abfss://sales@datalake11122000.dfs.core.windows.net", # contrainer@storageacc
mount_point = "/mnt/supermarket",
extra_configs = configs)
  

True

In [0]:
%fs

ls "/mnt/supermarket"


path,name,size,modificationTime
dbfs:/mnt/supermarket/Items.csv,Items.csv,16597,1696404193000
dbfs:/mnt/supermarket/Sales.csv,Sales.csv,51794172,1696402556000
dbfs:/mnt/supermarket/transformed-data/,transformed-data/,0,1696483183000


In [0]:
spark

In [0]:
sales=spark.read.format("csv").option("inferSchema","true").option("header","true").load("/mnt/supermarket/Sales.csv")

In [0]:
items=spark.read.format("csv").option("inferSchema","true").option("header","true").load("/mnt/supermarket/Items.csv")

In [0]:
sales.show(5)

+----------+--------------------+---------------+--------------------+---------------------------+--------------+-----------------+
|      Date|                Time|      Item Code|Quantity Sold (kilo)|Unit Selling Price (RMB/kg)|Sale or Return|Discount (Yes/No)|
+----------+--------------------+---------------+--------------------+---------------------------+--------------+-----------------+
|2020-07-01|2023-10-04 09:15:...|102900005117056|               0.396|                        7.6|          sale|               No|
|2020-07-01|2023-10-04 09:17:...|102900005115960|               0.849|                        3.2|          sale|               No|
|2020-07-01|2023-10-04 09:17:...|102900005117056|               0.409|                        7.6|          sale|               No|
|2020-07-01|2023-10-04 09:19:...|102900005115823|               0.421|                       10.0|          sale|               No|
|2020-07-01|2023-10-04 09:20:...|102900005115908|               0.539|      

In [0]:
items.show(5)

+---------------+--------------------+-------------+--------------------+
|      Item Code|           Item Name|Category Code|       Category Name|
+---------------+--------------------+-------------+--------------------+
|102900005115168|    Niushou Shengcai|   1011010101|Flower/Leaf Veget...|
|102900005115199|   Sichuan Red Cedar|   1011010101|Flower/Leaf Veget...|
|102900005115625|Local Xiaomao Cab...|   1011010101|Flower/Leaf Veget...|
|102900005115748|        White Caitai|   1011010101|Flower/Leaf Veget...|
|102900005115762|            Amaranth|   1011010101|Flower/Leaf Veget...|
+---------------+--------------------+-------------+--------------------+
only showing top 5 rows



In [0]:
items.groupBy("Category Name").agg(count("Item Name").alias("Count")).show()

+--------------------+-----+
|       Category Name|Count|
+--------------------+-----+
|            Capsicum|   45|
|Flower/Leaf Veget...|  100|
|             Cabbage|    5|
|     Edible Mushroom|   72|
|             Solanum|   10|
|Aquatic Tuberous ...|   19|
+--------------------+-----+



In [0]:
result=sales.join(items,on="Item Code",how="inner").select("Date","Time","Item Name","Category Name","Quantity Sold (kilo)","Unit Selling Price (RMB/kg)")

In [0]:
#Total Sales of each proudct year wise
result=result.withColumn("TotalSales",col("Quantity Sold (kilo)")*col("Unit Selling Price (RMB/kg)"))
total=result.withColumn("year",year("Date")).groupBy("Item Name","Year").agg(sum("TotalSales").alias("TotalSales"),first("Category Name").alias("Category Name"))

total.orderBy("Item Name","Year").show()

+--------------------+----+------------------+--------------------+
|           Item Name|Year|        TotalSales|       Category Name|
+--------------------+----+------------------+--------------------+
| 7 Colour Pepper (1)|2020| 2156.588599999999|            Capsicum|
| 7 Colour Pepper (1)|2021|1565.5511999999997|            Capsicum|
| 7 Colour Pepper (1)|2022| 553.1342000000003|            Capsicum|
| 7 Colour Pepper (1)|2023|1282.7123999999997|            Capsicum|
| 7 Colour Pepper (2)|2022|2280.3817000000004|            Capsicum|
| 7 Colour Pepper (2)|2023| 6700.024899999977|            Capsicum|
|7 Colour Pepper (...|2022|              91.0|            Capsicum|
|Agaricus Bisporus...|2022|              79.8|     Edible Mushroom|
|Agaricus Bisporus...|2022| 7932.899999999983|     Edible Mushroom|
|Agaricus Bisporus...|2023|14671.999999999545|     Edible Mushroom|
|               Aihao|2022|3.8160000000000003|Flower/Leaf Veget...|
|               Aihao|2023|39.504000000000005|Fl

In [0]:
#Total sales of each category month wise
Cat_Wise_sales=result.withColumn("TotalSales",round(col("Quantity Sold (kilo)")*col("Unit Selling Price (RMB/kg)"),2)).withColumn("year_month", date_format(to_date("Date", "yyyy-MM-dd"), "yyyy-MM")).groupBy("Category Name","year_month").agg(sum("TotalSales").alias("TotalSales"))
Cat_Wise_sales.orderBy("Category Name","year_month").show()

+--------------------+----------+------------------+
|       Category Name|year_month|        TotalSales|
+--------------------+----------+------------------+
|Aquatic Tuberous ...|   2020-07| 4481.130000000006|
|Aquatic Tuberous ...|   2020-08| 8731.340000000015|
|Aquatic Tuberous ...|   2020-09| 6520.800000000011|
|Aquatic Tuberous ...|   2020-10|10626.320000000005|
|Aquatic Tuberous ...|   2020-11| 7515.950000000008|
|Aquatic Tuberous ...|   2020-12|14757.269999999904|
|Aquatic Tuberous ...|   2021-01|17416.909999999836|
|Aquatic Tuberous ...|   2021-02|23360.199999999862|
|Aquatic Tuberous ...|   2021-03| 9402.719999999978|
|Aquatic Tuberous ...|   2021-04| 5638.600000000015|
|Aquatic Tuberous ...|   2021-05| 3331.430000000007|
|Aquatic Tuberous ...|   2021-06| 4693.199999999999|
|Aquatic Tuberous ...|   2021-07| 6891.520000000009|
|Aquatic Tuberous ...|   2021-08|10253.900000000007|
|Aquatic Tuberous ...|   2021-09| 9893.239999999978|
|Aquatic Tuberous ...|   2021-10|  9703.020000

In [0]:
Cat_Wise_sales.printSchema()

root
 |-- Category Name: string (nullable = true)
 |-- Month_Year: string (nullable = true)
 |-- TotalSales: double (nullable = true)



In [0]:
window_Spec=Window.partitionBy("Category Name").orderBy("year_month")
df = Cat_Wise_sales.withColumn("Sales_diff_prev", coalesce(col("TotalSales") - lag(col("TotalSales"), 1).over(window_Spec),col("TotalSales")))
df.show()

+--------------------+----------+------------------+-------------------+
|       Category Name|year_month|        TotalSales|    Sales_diff_prev|
+--------------------+----------+------------------+-------------------+
|Aquatic Tuberous ...|   2020-07| 4481.130000000006|  4481.130000000006|
|Aquatic Tuberous ...|   2020-08| 8731.340000000015|  4250.210000000009|
|Aquatic Tuberous ...|   2020-09| 6520.800000000011|-2210.5400000000036|
|Aquatic Tuberous ...|   2020-10|10626.320000000005|  4105.519999999994|
|Aquatic Tuberous ...|   2020-11| 7515.950000000008| -3110.369999999997|
|Aquatic Tuberous ...|   2020-12|14757.269999999904|  7241.319999999896|
|Aquatic Tuberous ...|   2021-01|17416.909999999836|  2659.639999999932|
|Aquatic Tuberous ...|   2021-02|23360.199999999862|  5943.290000000026|
|Aquatic Tuberous ...|   2021-03| 9402.719999999978|-13957.479999999885|
|Aquatic Tuberous ...|   2021-04| 5638.600000000015|-3764.1199999999626|
|Aquatic Tuberous ...|   2021-05| 3331.430000000007

In [0]:
result.write.mode("overwrite").csv("/mnt/supermarket/transformed-data/total_Sales")

In [0]:
total.repartition(1).write.mode("overwrite").option("header","true").csv("/mnt/supermarket/transformed-data/Yearly_Sales")

In [0]:
df.repartition(1).write.mode("overwrite").option("header","true").csv("/mnt/supermarket/transformed-data/Category_MonthSales")